# Process Orders Data
1. Ingest the data into the data lakehouse: bronze_orders
2. Perform Data quality checks and transform the data as required: silver_orders_clean
3. Explode the items array from the orders object: silver_orders

##### 1. Ingest the data into the data lakehouse: bronze_orders

In [0]:
create or refresh streaming table bronze_orders
comment 'Raw ingestion for orders data bronze'
tblproperties ('quality' = 'bronze')
as
select 
*,
_metadata.file_path as Input_file_path,
current_timestamp as ingestion_time
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/orders/',
  'json',
  map('cloudFiles.inferColumnTypes','true')
);

##### 2. Perform data quality checks and transform the data as required: silver_orders_clean

In [0]:
create or refresh streaming table silver_orders_clean(
constraint valid_customer_id expect (customer_id is not null) on violation fail update,
constraint valid_order_id expect (order_id is not null) on violation fail update,
constraint valid_orders_status expect (order_status in('Pending','Shipped','Cancelled','Completed')),
constraint valid_paayment_method expect (payment_method in('Credit Card', 'Bank Transfer', 'Paypal'))
)
comment 'Cleaned orders data'
tblproperties ('quality' = 'silver')
as
select 
order_id,
customer_id,
cast(order_timestamp as timestamp) as order_timestamp,
payment_method,
items,
order_status
from stream(live.bronze_orders);

##### 3. Explode the items array from the orders object: silver_orders

In [0]:
create streaming table silver_orders
as 
select 
order_id,
customer_id,
order_timestamp,
payment_method,
order_status,
items.item_id,
items.name as item_name,
items.price as item_price,
items.quantity as item_quantity,
items.category as item_category
from (
  select order_id, customer_id, order_timestamp, payment_method, order_status, explode(items) as items
  from stream(live.silver_orders_clean)
)